# **Functional Programming - Category Theory**

This is based on the book "Learning Functionl Programming" (Jack Widman).

> This notebook is quite technical, you can skip it.

## 1. Category Theory in Programming

We are not going to explain much about Category Theory, because we already
have other notes on it.

Just remember that a category has objects and morphisms. In programming,
our objects are types and morphisms are functions. This forms our category 
of programming.

In our category of programming, types are objects, and functions are morphisms.
What about values of a certain type? In Category Theory we cannot peak inside an object,
hence, we cannot talk about an instance of a type. Yet, there is a way around this.

First, any type with only one possible value is a terminal object in our category.
Thus, `Nothing` and `Tuple{}` are examples of terminal objects. A function
from a terminal type (object) to our type `T` is the same (isomorphic) to picking
a value of type `T`. Thus, in a weird way,
values are also morphisms. You can think of the value `1 :: Int` as the function
`1(x::Nothing)::Int`.

By thinking of values as functions themselves, we've made functions and values both "one-class citizens",
i.e. they are, in a sense, of the same computational class.

Also, running a function `f(x)` is equivalent to function compostiion, i.e. `f ∘ x`.
<!-- What about a function with multiple arguments? Just turn values into a tuple:
`f(x,y)` ≅ `f ∘ tuple  -->

In [5]:
using Pkg
Pkg.activate(".")

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/FunctionalProgramming`


## 2. Functors

A functor goes from a category to another. In programming,
we usually have "endofunctors", which are functors that go
from a category to itself. In our case, it's the category of programming.

Thus, a functor `F` must take a type `T` and return another type `FT`.
Also, `F` must take a function `f(x::T)::T'` and return
a function `Ff( Fx :: FT)::FT'`.

Remember also that a functor is covariant, i.e. $F(f \circ g) = F(f) \circ F(g)$.
Also, it takes identities to identities, i.e. $F(id_A) = id_{FA}$.

> :warning: The definition of a functor in programming usually does not
define `F(f)` returning a function, but instead it codes
`fmap(f,F(a))` which is the same as `F(f)(F(a))`.

Functors act on types and morphisms. But as we've shown above, a value of type `T` can also
be thought of as a morphism from a termminal type to type `T`.

This means that `F(x::T)` ≅ `F(x(::Nothing)::T)`.
Therefore `F(x): F(Nothing) → F(T)`.

Let's do an example. Remember that a functor must take types to types,
and functions to functions.
Since a value can be thought as a function, let's also extend our functor to act on values.

So our functor is a bit different from the Category Theory definition. It acts on types
to types (object to objects), from functions to functions (morphisms to morphisms) and
values to values, which are isomorphic to morphisms, so we are theoretically safe.

Let's start with the following functor `F`.

In [6]:
struct MyType
    x
end


# Defining a simple functor for MyType
F(T::Type) = MyType
@show F(String) == MyType
@show typeof(F(String));
F(f::Function) = a::MyType -> F(f(a.x))

# Isomorphism
# F(x) = x::MyType -> MyType(x)
F(x) = MyType(x)



f(x::Int)::String = string(x)
g(s::String)::Int = length(s)

Ff = F(f);
Fg = F(g);

@show F((f ∘ g))(F("ok")) == (F(f) ∘ F(g))(F("ok"))
@show F(identity(1)) == identity(F(1));
println("----------------------------")

F(String) == MyType = true
typeof(F(String)) = DataType
(F(f ∘ g))(F("ok")) == (F(f) ∘ F(g))(F("ok")) = true
F(identity(1)) == identity(F(1)) = true
----------------------------



First, by declaring a `struct MyType`  we are creating a new type (object) `MyType`.
Let's now create a functor that takes every object in our category to the object `MyType`.
Every function `f(x::T)::T'` must now become a function `Ff(Fx::MyType)::MyType` and
must preserve composition.


Since a value `x` is a function `x(::Nothing)::T`, then `F(x)` must be a function
`Fx(a::MyType)::MyType`.
`MyType` has only one field `x`, this means that every instance of this type is actually
isomorphic to a function `a ::MyType-> MyType(x)`. Hence, we can drop all this shinnenigans and just write
`F(x) = MyType(x)`.

Using this contruction, we can show that this commutes properly,
so we have indeed defined a functor.

### Name Overloading

In our example we separated the type `MyType` from the functor `F` associated to it.
Indeed, types and functors do not need to be associated.

Note that in our original example, `S` was a functor that sent everything to `String`
and every function to the trivial `x->""`. We could define many other functors, such
as `S2` which would take `x::String->"2"`.

## 3. More Examples of Functors

As we've said before, it's actually more common to define function via a `map` or `fmap`
high order function. This `fmap` takes a function and "lifts" it, i.e.
lifts `f(a)` to `F(f)(F(a))`.
Also, it's more common to define functors by using parametric structs.
For example:

In [7]:
struct G{T}
    x::T
end
G(T::Type) = G{T}
fmap(f::Function, a::G{T}) where T = G(f(a.x))
G(f::Function) = a -> fmap(f,a);

In [8]:
@show G(identity(1)) == identity(G(1))
@show G(f ∘ g)(G("test")) == (G(f) ∘ G(g))(G("test"));

G(identity(1)) == identity(G(1)) = true
(G(f ∘ g))(G("test")) == (G(f) ∘ G(g))(G("test")) = true


Note that, in our example, `G` is the functor, and it takes a type `T` and returns a type `G{T}`. 

#### Maybe/Option Function 

A very common example of functor is the so called `Maybe` or `Option` fuctor.
This functor takes types `T` to `Maybe{T}`.
The type `Maybe{T}`contains values of type
`Nothing` and of type `Just{T}`.

In [24]:
struct Just{T}
    x::T
end
just(J::Just) = J.x

Maybe{T} = Union{Nothing,Just{T}}
Maybe(T::Type) = Union{Nothing,Just{T}}
Maybe(a::T) where T = Just(a)
Maybe(::Nothing) = nothing
Maybe() = nothing

fmap(f::Function, a::Just{T}) where T = Just(f(a.x))
fmap(f::Function, a::Nothing) = nothing


Maybe(f::Function) = a::Maybe -> fmap(f,a)

Maybe (alias for Union{Nothing, Just{T}} where T)

In [25]:
Maybe(identity)(Maybe())

f(x::Int)::Int = x^2
g(x::Int)::Int = x + 1

Maybe(f∘g)(Maybe(10)) == (Maybe(f) ∘ Maybe(g))(Maybe(10))

true

In [27]:
Maybe(identity)(Maybe()) isa Nothing

true

### Example - Using our Maybe Functor

Let's do an use example. We have a function `getuserfromdb` that get's a value from a database
by searching the index. If the index is not in the database, it returns nothing.
Hence, our function is of type `Union{User, Nothing}`.

Next, we have a funciton `getname(x::User)::String`, that returns the
name of the user, if a user is passed.
We cannot (in good concious) compose our function `getname` with the function
`getuserfromdb`, because the types don't match.

We could solve this by extending our `getname` function to be able to receive null values,
then apply an if statement to check if the null is passed, and then return null... But this does
not sound nice. Our `getname` does exactly what's suppose to do. We don't want to add some
extra commands to deal with incorrect input.

Hence, we can use the `Maybe` functor. Using `Maybe(getname)`
we have a new function that takes `Maybe{User}` and returns `Maybe{String}`.

In [11]:
struct User
    name::String
end
DATABASE = Dict( 4 =>User("James"), 2 =>User("John"), 1 =>User("Peter"));

In [12]:
function getuserfromdb(uid::Int)::Union{User, Nothing}
    if uid in keys(DATABASE)
        return DATABASE[uid]
    end
    return nothing
end

getuserfromdb(1)

User("Peter")

In [13]:
@show Maybe(getuserfromdb(10)) isa Maybe{User}
@show Maybe(getuserfromdb(1)) isa Maybe{User};

Maybe(getuserfromdb(10)) isa Maybe{User} = true
Maybe(getuserfromdb(1)) isa Maybe{User} = true


In [14]:
getname(x::User)::String = x.name

getname (generic function with 1 method)

In [15]:
@show Maybe(getname)(Maybe(getuserfromdb(10)))
@show fmap(getname, Maybe(getuserfromdb(10)))
@show fmap(getname, Maybe(getuserfromdb(1)));

(Maybe(getname))(Maybe(getuserfromdb(10))) = Nothing
fmap(getname, Maybe(getuserfromdb(10))) = Nothing
fmap(getname, Maybe(getuserfromdb(1))) = Just{String}("Peter")
